In [7]:
import pandas as pd
import json
import re

In [8]:
# Load JSON data from file
with open('reddit_posts.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [9]:
# Convert JSON to DataFrame
df = pd.json_normalize(data)

# Convert `created_utc` to datetime
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)

# Convert list columns to comma-separated strings
df['mentioned_countries'] = df['mentioned_countries'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
df['mentioned_cities'] = df['mentioned_cities'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

# Display DataFrame
df

,submission_id,author,created_utc,month_year,name,num_comments,score,selftext,subreddit_name,title,upvote_ratio,mentioned_countries,mentioned_cities
0,1jjuq8t,Stock_Rabbit_1901,2024-04-09 12:57:21.441380978+00:00,04-2024,t3_1jjuq8t,5,6,I’m going to China in May to visit a manufactu...,travel,"China trip, advice for 6 nights",1.00,china,"beijing, hangzhou, shanghai"
1,1jh273f,Wit_in_the_Wild,2024-07-11 15:20:07.773072004+00:00,07-2024,t3_1jh273f,104,323,"31F. Been a long-time lurker here, and with my...",solotravel,"Fence-sitters, go take that leap! - Quit my jo...",0.90,"singapore, malaysia, thailand, china","bangkok, hong kong, kuala lumpur, singapore"
2,1j62q8v,OkMasterpiece260,2022-04-27 16:00:12.938550949+00:00,04-2022,t3_1j62q8v,6,0,Hey backpackers! My name is Caitlin and I’m cu...,backpacking,Come to Thailand with me!!,0.09,"singapore, thailand",singapore
3,1jcrmy9,Extension-Driver-136,2024-12-25 09:13:10.947906971+00:00,12-2024,t3_1jcrmy9,23,13,"Hello everyone, \n\nI would like some help ple...",solotravel,Budgeting for Gap Year in SEA - Help please,0.84,"vietnam, malaysia, thailand, indonesia, nepal,...",
4,1eammc7,Loud-Gap8196,2022-11-28 14:02:49.658598900+00:00,11-2022,t3_1eammc7,0,0,,wanderlust,Luxury on a Budget! The Island Life on Koh Sam...,0.25,thailand,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,1d3iy93,travelwithme24x7,2023-05-07 03:14:31.708024025+00:00,05-2023,t3_1d3iy93,0,1,,wanderlust,Best Things To Do In Istanbul Turkey (Itinerar...,1.00,turkey,istanbul
1309,1ihy9c2,None,2022-09-02 19:40:51.985013008+00:00,09-2022,t3_1ihy9c2,46,3,"Hey everyone, I am hoping to do a little bit o...",solotravel,Which option would you choose for your first m...,0.60,"singapore, malaysia, thailand, indonesia, chin...","bali, hong kong, kyoto, osaka, seoul, singapor..."
1310,1ip7sjm,lifeisabop,2022-08-27 11:58:04.199906111+00:00,08-2022,t3_1ip7sjm,2,1,Hi everyone! Am in a bit of an interesting sit...,solotravel,Accidentally granted two Vietnam eVisas?,1.00,vietnam,
1311,1ig4d0y,None,2022-11-15 01:36:19.114248991+00:00,11-2022,t3_1ig4d0y,2,1,"Hello,\n\nI am planning on making a trip to Vi...",solotravel,Vietnam Itinerary - few concerns,1.00,"vietnam, philippines","hanoi, manila"


In [10]:
# Drop duplicates (avoid biased sentiment analysis)
df = df.drop_duplicates(keep="first")

In [11]:
# Convert 'created_utc' to datetime format
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)

# Split into separate 'date' and 'time' columns
df['date_only'] = df['created_utc'].dt.date  # Extracts only the date (YYYY-MM-DD)
df['time_only'] = df['created_utc'].dt.time  # Extracts only the time (HH:MM:SS)


# Display the updated DataFrame with new columns
print(df[['created_utc', 'date_only', 'time_only']])

# # Extract day of the week (0 = Monday, 6 = Sunday)
# df['day_of_week'] = df['created_utc'].dt.dayofweek

# # Extract month
# df['month'] = df['created_utc'].dt.month

# # Extract year
# df['year'] = df['created_utc'].dt.year

# # Extract whether the post was made on a weekend (1 if weekend, 0 if weekday)
# df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# # Extract weekday name (e.g., Monday, Tuesday)
# df['weekday_name'] = df['created_utc'].dt.strftime('%A')

# # Display the updated DataFrame with new columns
# print(df[['created_utc', 'date_only', 'time_only', 'day_of_week', 'month', 'year', 'is_weekend', 'weekday_name']])

                             created_utc   date_only        time_only
0    2024-04-09 12:57:21.441380978+00:00  2024-04-09  12:57:21.441380
1    2024-07-11 15:20:07.773072004+00:00  2024-07-11  15:20:07.773072
2    2022-04-27 16:00:12.938550949+00:00  2022-04-27  16:00:12.938550
3    2024-12-25 09:13:10.947906971+00:00  2024-12-25  09:13:10.947906
4    2022-11-28 14:02:49.658598900+00:00  2022-11-28  14:02:49.658598
...                                  ...         ...              ...
1308 2023-05-07 03:14:31.708024025+00:00  2023-05-07  03:14:31.708024
1309 2022-09-02 19:40:51.985013008+00:00  2022-09-02  19:40:51.985013
1310 2022-08-27 11:58:04.199906111+00:00  2022-08-27  11:58:04.199906
1311 2022-11-15 01:36:19.114248991+00:00  2022-11-15  01:36:19.114248
1312 2022-12-19 21:46:04.060967922+00:00  2022-12-19  21:46:04.060967

[1313 rows x 3 columns]


In [12]:
from langdetect import detect, LangDetectException

# Function to detect language
def detect_language(text):
    try:
        if isinstance(text, str) and text.strip(): # Check if text is a non-empty string
            return detect(text)
        else:
            return None # Handle cases with non-string or empty text
    except LangDetectException:
        return None  # Handle language detection errors

# Apply language detection to 'selftext' column
df['language'] = df['selftext'].apply(detect_language)

# Filter out rows that are not English
non_english_rows = df[df['language'] != 'en']

# Display non-English rows
print("Non-English Rows:")
print(non_english_rows[['selftext', 'language']])

# Count the number of non-English rows
non_english_count = len(non_english_rows)
print(f"\nNumber of non-English rows: {non_english_count}")

Non-English Rows:
     selftext language
4                 None
8                 None
19                None
21                None
29                None
...       ...      ...
1281              None
1300              None
1303              None
1305              None
1308              None

[131 rows x 2 columns]

Number of non-English rows: 131


In [13]:
def clean_text(text):
    # Remove emojis and other unwanted symbols
    text = re.sub(r'[^\w\s,.!?;]', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove extra spaces and trim
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Example usage
df['cleaned_caption'] = df['selftext'].apply(clean_text)

# Display cleaned captions
print(df[['selftext', 'cleaned_caption']])

                                               selftext  \
0     I’m going to China in May to visit a manufactu...   
1     31F. Been a long-time lurker here, and with my...   
2     Hey backpackers! My name is Caitlin and I’m cu...   
3     Hello everyone, \n\nI would like some help ple...   
4                                                         
...                                                 ...   
1308                                                      
1309  Hey everyone, I am hoping to do a little bit o...   
1310  Hi everyone! Am in a bit of an interesting sit...   
1311  Hello,\n\nI am planning on making a trip to Vi...   
1312  Hey everyone,\n\nI'm currently refining my act...   

                                        cleaned_caption  
0     Im going to China in May to visit a manufactur...  
1     31F. Been a longtime lurker here, and with my ...  
2     Hey backpackers! My name is Caitlin and Im cur...  
3     Hello everyone, I would like some help please!...  
4

In [ ]:
from googletrans import Translator

def analyze_and_translate_languages_with_google(dataset):
    # Initialize the Google Translator
    translator = Translator()

    # Function to detect language using Google Translator
    def detect_language(text):
        try:
            return translator.detect(text).lang
        except Exception as e:
            return 'unknown'

    # Function to translate text to English
    def translate_to_english(text):
        try:
            return translator.translate(text, dest='en').text
        except Exception as e:
            return None

    # List to keep track of rows to drop
    to_drop = []

    # Iterate over the dataset and apply the logic for detecting and translating languages
    for index, row in dataset.iterrows():
        detected_language = detect_language(row["cleaned_caption"])

        if detected_language == 'en':
            dataset.at[index, "language"] = 'en'
            continue  # If already English, move to the next caption
        elif detected_language == 'unknown':
            to_drop.append(index)  # Drop the row if language is unknown
        else:
            # Translate to English
            translated_text = translate_to_english(row["cleaned_caption"])
            if translated_text:
                # Re-detect language after translation
                new_language = detect_language(translated_text)
                if new_language == 'en':
                    # Update with the translated text and mark the language as English
                    dataset.at[index, "cleaned_caption"] = translated_text
                    dataset.at[index, "language"] = 'en'
                else:
                    to_drop.append(index)  # Drop if still not English
            else:
                to_drop.append(index)  # Drop if translation failed

    # Drop rows where the language is still unknown or non-English
    dataset.drop(to_drop, inplace=True)

    return dataset  # Return the cleaned dataset with only English caption

# Example usage with your dataset
translated_caption = analyze_and_translate_languages_with_google(df)
print(translated_caption[['cleaned_caption', 'language']])

In [ ]:
# df.describe()

In [ ]:
# do sentiment analysis and give sentiment scores using vader nlp
import nltk
# nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis
translated_caption["sentiment_score"] = translated_caption["cleaned_caption"].apply(lambda text: analyzer.polarity_scores(text)["compound"])

# Display the results
print(df.head())

In [ ]:
translated_caption.to_json("sentiment_analysis(reddit).json", orient="records", date_format="iso")
